In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HF_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain.chat_models import init_chat_model

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq"

)


# Summarization MiddleWare

In [2]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage


In [3]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

# model used for summarization
summarizer_model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq"
)

# agent model
agent_model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq"
)

agent = create_agent(
    model=agent_model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=summarizer_model,
            checkpoint=InMemorySaver(),
            trigger=("messages", 1),   # ← plural
            keep_history=("messages", 1)
        )
    ]
)

config = {"configurable": {"thread_id": "thread_1"}}

questions = [
    "what is the capital of France?",
    "what is the capital of Germany?",
    "what is the capital of Italy?",
    "what is the population of France?",
    "what is the population of Germany?",
    "what is the population of Italy?"
]


for q in questions:
    response = agent.invoke(
        {"messages": [HumanMessage(content=q)]},
        config=config
    )

    last_msg = response["messages"][-1].content
    print("Message:", last_msg)
    print("Length:", len(last_msg))



Message: The capital of France is Paris.
Length: 31
Message: The capital of Germany is Berlin.
Length: 33
Message: The capital of Italy is Rome.
Length: 29
Message: As of my cut-off knowledge in 2023, the estimated population of France is approximately 67.2 million people.
Length: 108
Message: As of my cut-off knowledge in 2023, the estimated population of Germany is approximately 83.2 million people.
Length: 109
Message: As of my cut-off knowledge in 2023, the estimated population of Italy is approximately 59.2 million people.
Length: 107


# Human in the Loop Middleware

In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model

def read_email(email_id: str) -> str:
    """Read the content of an email by its ID."""
    return f"Email {email_id} content"


def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient with subject and body."""
    return f"Email Sent to {recipient} with subject '{subject}'"


agent_model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
)


In [5]:
agent = create_agent(
    model=agent_model,
    tools=[read_email, send_email],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {
                    "allowed_decisions": ["approve", "edit", "reject"]
                },
                "read_email": False,
            }
        )
    ],
)

In [8]:
config = {"configurable": {"thread_id": "thread_2"}}

result = agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="Send Email to nithin@gmail.com with subject 'hello' body 'how are you'"
            )
        ]
    },
    config=config,
)


In [9]:
from langgraph.types import Command

result = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "tool_name": "send_email",
                    "type": "approve"
                }
            ]
        }
    ),
    config=config
)


In [ ]:
result

{'messages': [HumanMessage(content="Send Email to nithin@gmail.com with subject 'hello' body 'how are you'", additional_kwargs={}, response_metadata={}, id='762d347e-84cb-43b0-962e-cafa81be8b21'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm45mn84fp', 'function': {'arguments': '{"body":"how are you","recipient":"nithin@gmail.com","subject":"hello"}', 'name': 'send_email'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 314, 'total_tokens': 347, 'completion_time': 0.038593067, 'completion_tokens_details': None, 'prompt_time': 0.020819705, 'prompt_tokens_details': None, 'queue_time': 0.047095855, 'total_time': 0.059412772}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c36e0-577d-77b2-8cea-fe4f9411354c-0', tool_calls=[{'name': 'send_email', 'args': {'body': 'how ar

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model

def read_email(email_id: str) -> str:
    """Read the content of an email by its ID."""
    return f"Email {email_id} content"


def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient with subject and body."""
    return f"Email Sent to {recipient} with subject '{subject}'"


agent_model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
)


agent = create_agent(
    model=agent_model,
    tools=[read_email, send_email],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {
                    "allowed_decisions": ["approve", "edit", "reject"]
                },
                "read_email": False,
            }
        )
    ],
)

config = {"configurable": {"thread_id": "thread_312"}}

result = agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="Send Email to nithin@gmail.com with subject 'hello' body 'how are you'"
            )
        ]
    },
    config=config,
)


from langgraph.types import Command

result = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "tool_name": "send_email",
                    "type": "reject"
                }
            ]
        }
    ),
    config=config
)
result
